In [78]:
import pandas as pd
import spacy
nlp = spacy.load('en_core_web_sm')

In [79]:
data = pd.read_json('restaurant.json')
data.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
100086,h8l10hiVsn4DlAv3_LlsQw,UFE_5r4ewNK3jA2pRPn2ww,r5PLDU-4mSbde5XekTXSCA,5,0,0,0,Wow! Really good Italian food with a grocery s...,2018-07-08 22:42:32
1013,vvIzf3pr8lTqE_AOsxmgaA,MAmijW4ooUzujkufYYLMeQ,r5PLDU-4mSbde5XekTXSCA,4,0,0,0,We have been trying Eggplant sandwiches all ov...,2015-04-15 04:50:56
101755,27t2Z9QXd6Pm9lKfEG_LzQ,kjeX2RXvW7RhBbD2QLd5jA,r5PLDU-4mSbde5XekTXSCA,5,7,8,6,"(Lyrics - Falco - Rock Me Amadeus)\n\nOoh, Roc...",2014-01-20 16:31:22
102164,ZB4Okiod5Yxaxx5UEEvdWg,fDBybzZAL5UDscd33HCXyA,r5PLDU-4mSbde5XekTXSCA,4,3,4,2,The omnipresent crowds here speak volumes abou...,2015-12-06 23:21:51
102256,BbJWlQRUPGFxnlZbgpdrLA,62JJoUPxKxqb6snMJxi2ng,r5PLDU-4mSbde5XekTXSCA,3,0,0,0,I had a bruschetta open face sandwich basicall...,2017-12-18 22:42:51


In [80]:
menu = ["Cheese Steak", "Cheesesteak", "Steak and Cheese", "Italian Combo", "Tiramisu", "Cannoli",
        "Chicken Salad", "Chicken Spinach Salad", "Meatball", "Pizza", "Pizzas", "Spaghetti",
        "Bruchetta", "Eggplant", "Italian Beef", "Purista", "Pasta", "Calzones",  "Calzone",
        "Italian Sausage", "Chicken Cutlet", "Chicken Parm", "Chicken Parmesan", "Gnocchi",
        "Chicken Pesto", "Turkey Sandwich", "Turkey Breast", "Ziti", "Portobello", "Reuben",
        "Mozzarella Caprese",  "Corned Beef", "Garlic Bread", "Pastrami", "Roast Beef",
        "Tuna Salad", "Lasagna", "Artichoke Salad", "Fettuccini Alfredo", "Chicken Parmigiana",
        "Grilled Veggie", "Grilled Veggies", "Grilled Vegetable", "Mac and Cheese", "Macaroni",  
         "Prosciutto", "Salami"]

In [81]:
indexOfTheData = 14
text_doc = nlp(data.text.iloc[indexOfTheData])
print(text_doc)

I had the cheesesteak sub my husband had the chicken Parmesan sub. Both were delicious but mine tasted better. Portion size: We each ordered the small one so we could save room for dessert. The sandwich covered most of a normal sized dinner plate and we had to take dessert with us.


In [82]:
# for token in text_doc:
#     print(f"{token}\t\t{token.lemma_}\t\t{token.is_stop}")

In [83]:
from spacy.matcher import PhraseMatcher
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
patterns = [nlp(item) for item in menu]
matcher.add("MENUS", None, *patterns)
matches_found = matcher(text_doc)
print(matches_found)
for i in matches_found:
    print(text_doc[i[1]: i[2]])

[(1333928641185522206, 3, 4), (1333928641185522206, 9, 11)]
cheesesteak
chicken Parmesan


## Matching throughout the whole data set
       

In [90]:
from collections import defaultdict
item_ratings = defaultdict(list)

In [95]:
for idx, review in data.iterrows():
    doc = nlp(review.text)
    matches = matcher(doc)
    
    found_items = set([ doc[ match[1]:match[2] ] for match in matches])
    
    for item in found_items:
        item_ratings[str(item).lower()].append(review.stars)


In [97]:
# print(item_ratings)

In [107]:
# mean_ratings_old = {item: sum(ratings)//len(ratings) for item, ratings in item_ratings.items()}

mean_ratings = {}
for item, ratings in item_ratings.items():
    mean_ratings[item]=sum(ratings)/len(ratings)


In [105]:
worst_item = sorted(mean_ratings, key=mean_ratings.get)[0]

In [108]:
print(worst_item)

eggplant


In [113]:
counts = {item: len(ratings) for item, ratings in item_ratings.items()}
# counts = {}
# for item, ratings in item_ratings.items():
#     counts[item] = len(ratings)
# print(counts)
item_counts = sorted(counts, key=counts.get, reverse=True)
for item in item_counts:
    print(f"{item:>25}\t{counts[item]:>5}")

                    pizza	  716
                    pasta	  510
                 meatball	  326
              cheesesteak	  292
                  calzone	  220
                 eggplant	  190
                  cannoli	  178
             cheese steak	  176
                  lasagna	  166
                  purista	  134
               prosciutto	  126
             chicken parm	  116
          italian sausage	  114
             garlic bread	   92
                  gnocchi	   90
                spaghetti	   82
                 calzones	   76
                   pizzas	   66
                   salami	   64
            chicken pesto	   60
             italian beef	   58
                 tiramisu	   54
                     ziti	   52
            italian combo	   44
         chicken parmesan	   42
       chicken parmigiana	   36
           mac and cheese	   36
               portobello	   36
                 pastrami	   32
           chicken cutlet	   22
         steak and cheese	   18
        

In [114]:
sorted_ratings = sorted(mean_ratings, key=mean_ratings.get)

print("Worst rated menu items:")
for item in sorted_ratings[:10]:
    print(f"{item:20} Ave rating: {mean_ratings[item]:.2f} \tcount: {counts[item]}")
    
print("\n\nBest rated menu items:")
for item in sorted_ratings[-10:]:
    print(f"{item:20} Ave rating: {mean_ratings[item]:.2f} \tcount: {counts[item]}")

Worst rated menu items:
chicken cutlet       Ave rating: 3.55 	count: 22
turkey sandwich      Ave rating: 3.80 	count: 10
spaghetti            Ave rating: 3.85 	count: 82
italian combo        Ave rating: 3.91 	count: 44
eggplant             Ave rating: 3.97 	count: 190
italian beef         Ave rating: 4.00 	count: 58
tuna salad           Ave rating: 4.00 	count: 10
garlic bread         Ave rating: 4.02 	count: 92
meatball             Ave rating: 4.08 	count: 326
portobello           Ave rating: 4.11 	count: 36


Best rated menu items:
prosciutto           Ave rating: 4.62 	count: 126
purista              Ave rating: 4.64 	count: 134
chicken salad        Ave rating: 4.67 	count: 12
pastrami             Ave rating: 4.69 	count: 32
reuben               Ave rating: 4.80 	count: 10
steak and cheese     Ave rating: 4.89 	count: 18
artichoke salad      Ave rating: 5.00 	count: 10
corned beef          Ave rating: 5.00 	count: 4
fettuccini alfredo   Ave rating: 5.00 	count: 12
turkey breast    